In [58]:
import os
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [70]:
names = os.listdir('./경남')

In [71]:
names = [name[:-4] for name in names]

In [ ]:
names

In [74]:
df_경상남도 = pd.DataFrame()

for name in names:
    df = pd.read_csv(f'./경상남도/{name}.csv',encoding='ANSI')
    df_경상남도 = pd.concat([df_경상남도,df],ignore_index=True)

==================================================================================================

==================================================================================================

==================================================================================================

In [ ]:
# 파일명 변경

folder_names = os.listdir('./경남/')

for folder_name in folder_names:
    file_names = os.listdir(f'./경남/{folder_name}')
    for file_name in file_names:
        split_file_name = file_name.replace('.csv','').split('_')
        os.rename(f'./경남/{folder_name}/{file_name}',f'./경남/{folder_name}/{folder_name}_{split_file_name[1]}.csv')

==================================================================================================

==================================================================================================

==================================================================================================

### 데이터프레임 합치기
### 관광지 기본정보

In [ ]:
basic_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./경남/{folder_name}/{folder_name}_관광지 기본정보.csv',encoding='ANSI')
    basic_info = pd.concat([basic_info,df],ignore_index=True)

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기

### 내비게이션

In [ ]:
navi_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./경남/{folder_name}/{folder_name}_내비게이션 목적지 검색량 추이.csv',encoding='ANSI',index_col=0)
    df = df.rename(columns={df.columns[0] : folder_name}).transpose()
    df['관광지명'] = folder_name
    navi_info = pd.concat([navi_info,df])

In [ ]:
order = ['관광지명',202301,202302, 202303, 202304,202305, 202306, 202307, 202308,202309, 202310, 202311, 202312]
navi_info = navi_info[order]

In [ ]:
navi_info.to_csv('./경남/navi_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기
### 방문자 거주지별 분포

In [ ]:
order = ['관광지명','광역지자체명', '기초지자체명', '광역지자체별 거주 방문자 비율', '기초지자체별 거주 방문자 비율']

visit_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./경남/{folder_name}/{folder_name}_방문자 거주지별 분포.csv',encoding='ANSI')
    df['관광지명'] = folder_name
    df = df[order]
    
    visit_info = pd.concat([visit_info,df],ignore_index=True)

In [ ]:
visit_info.to_csv('./경남/visit_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기
### 성별

In [ ]:
order = ['관광지명','연령대', '남성방문 비율', '여성방문 비율']

gender_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./경남/{folder_name}/{folder_name}_성·연령별 방문자.csv',encoding='ANSI')
    df['관광지명'] = folder_name
    df = df[order]
    gender_info = pd.concat([gender_info,df],ignore_index=True)

In [ ]:
gender_info.to_csv('./경남/gender_info.csv',encoding='utf-8',index=False)

### 데이터 가져오기
### 위치정보

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
url = 'https://datalab.visitkorea.or.kr/visualize/getTempleteData.do'
city_code = [48125,48127,48123,48121,48129]

all = []

for i in city_code:
    params = {
        "SIDO_CD": 29,
        "SGG_CD": i,
        "BASE_YM1": 2010,
        "BASE_YM2": 2022,
        "CONT_NM": "",
        "KTO_CATE_LCLS_CD": "",
        "KTO_CATE_MCLS_CD": "",
        "firstIndex": float('nan'),
        "pageIndex": 1,
        "recordCountPerPage": 10,
        "orderSelect": "",
        "orderValue": "asc",
        "contentList": "전체",
        "noTheme": "",
        "qid": "LN_05_01_011"
    }

    response = requests.get(url,params=params)
    data = response.json()
    
    for item in data.get('list', []):
        result = {
            '관광지명': item.get('CONT_NM'),
            '주소': item.get('TAR_ADDR_NM'),
            '대분류': item.get('KTO_CATE_LCLS_NM'),
            '중분류': item.get('KTO_CATE_MCLS_NM'),
            '소분류': item.get('KTO_CATE_SCLS_NM'),
            'longitude': item.get('XCRD'),
            'latitude': item.get('YCRD')
        }
        all.append(result)
        
    if len(all) == 101:
        break

In [ ]:
df = pd.DataFrame(all)

In [ ]:
df = pd.merge(basic_info,df,on='관광지명',how='outer')

In [ ]:
df.to_csv('./경남/location_info.csv',encoding='utf-8',index=False)

==================================================================================================

==================================================================================================

==================================================================================================

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

stars = []
reviews = []

for i in basic_info['관광지명']:
    url = f'https://www.google.co.kr/maps/search/{i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    
    try:
        # 해당 클래스명이 나타날 때까지 최대 10초 대기
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'F7nice'))
        )
        
        star = element.text.split('\n')[0]
        review = element.text.split('\n')[1][1:-1]
        
        stars.append(star)
        reviews.append(review)
    except:
        check = input('일치하는 장소를 클릭 후 y를 눌러주세요(이외 None)')
        if check.upper() == 'Y':
            star = driver.find_element(By.CLASS_NAME, 'F7nice').text.split('\n')[0]
            review = driver.find_element(By.CLASS_NAME, 'F7nice').text.split('\n')[1][1:-1]
            
            stars.append(star)
            reviews.append(review)
        else:
            stars.append(None)
            reviews.append(None)
    
    driver.quit()
    print(f'평점 수 : {len(stars)}','|',f'리뷰 수 : {len(reviews)}','|',f'관광지 명 : {i}','|', f'별점 : {star}','|',f'리뷰 : {review}')
    
    star = None
    review = None

In [ ]:
print(f'총 데이터 수 : {len(basic_info)}')
print('=========================================')
print(f'평점 수 : {len(stars)}')
print(f'리뷰 수 : {len(reviews)}')
print('=========================================')
print(f'총 None값 수 : {stars.count(None)}')
print(f'총 None값 수 : {reviews.count(None)}')

총 데이터 수 : 49
평점 수 : 49
리뷰 수 : 49
총 None값 수 : 2
총 None값 수 : 2


In [ ]:
basic_info['stars'] = stars
basic_info['reviews'] = reviews

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

### 리뷰, 체류시간 -> 숫자 값으로 변경

In [ ]:
time = []

for i in basic_info['평균 체류시간']:
    i = i.replace('시간','').replace('분','').strip()
    i = i.split(' ')
    time.append((int(i[0]) * 60) + int(i[1]))
    
basic_info['평균 체류시간'] = time

In [ ]:
reviews_to_int = []

for i in basic_info['reviews']:
    if type(i) == type(None):
        reviews_to_int.append(i)
        continue
    if type(i) in (float,int):
        reviews_to_int.append(i)
    else:
        i = i.replace(',','')
        i = int(i)
        reviews_to_int.append(i)

basic_info['reviews'] = reviews_to_int

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

### None값 -> 관광지별 평균

In [ ]:
navi_info.set_index('관광지명',inplace=True)

In [ ]:
navi_info = navi_info.apply(lambda row: row.fillna(int(row.mean())), axis=1)

In [ ]:
navi_info.to_csv('./경남/navi_info.csv',encoding='utf-8',index=False)

### 광역지자체별 거주 방문자 비율

In [ ]:
visit_info.dropna(subset=['광역지자체명'], axis=0,inplace=True)

In [ ]:
names = visit_info['관광지명'].tolist()

local_visit = visit_info[visit_info['광역지자체명'].str.contains('광주')].groupby('관광지명')[['광역지자체별 거주 방문자 비율']].mean()
local_visit = local_visit.reset_index()

local_visit.loc[local_visit['관광지명'].isin(names)]

In [ ]:
basic_info['광역지자체별 거주 방문자 비율'] = local_visit['광역지자체별 거주 방문자 비율']

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

===============================================================================================================

===============================================================================================================

===============================================================================================================

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

search_number = []
for i in basic_info['관광지명']:
    if not i.startswith('창원'):
        i = '창원 ' + i
    url = f'https://www.google.com/search?q={i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)

    search_number.append(driver.find_element(By.ID, 'result-stats').text)

basic_info['search_number'] = search_number

In [ ]:
search_number = []
for index,i in enumerate(basic_info['관광지명']):
    if basic_info.iloc[index]['search_number']:
        continue
    if not i.startswith('창원'):
        i = '창원 ' + i
    url = f'https://www.google.com/search?q={i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)

    # search_number.append(driver.find_element(By.ID, 'result-stats').text)
    basic_info.loc[index,'search_number'] = driver.find_element(By.ID, 'result-stats').text

In [ ]:
import re

pattern = re.compile(r'\([^)]*\)')

basic_info['search_number'] = basic_info['search_number'].apply(lambda x: re.sub(pattern,'',x).strip())

basic_info['search_number'] = basic_info['search_number'].apply(lambda x: int(x.replace('개','').replace(',','')[7:]))

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

===============================================================================================================

===============================================================================================================

===============================================================================================================

In [1]:
import pandas as pd

basic_info = pd.read_csv('./경남/basic_info.csv')
gender_info = pd.read_csv('./경남/gender_info.csv')
location_info = pd.read_csv('./경남/location_info.csv')
navi_info = pd.read_csv('./경남/navi_info.csv')
visit_info = pd.read_csv('./경남/visit_info.csv')

In [2]:
totals = []
avgs = []

for i in range(len(navi_info)):
    total = navi_info.iloc[i][1:].sum()
    avg = navi_info.iloc[i][1:].mean()
    
    totals.append(total)
    avgs.append(avg)
navi_info['sum'] = totals
navi_info['avg'] = avgs

In [3]:
df = pd.DataFrame()
df = pd.concat([basic_info,navi_info[['sum','avg']]],axis=1)
df = pd.merge(df,location_info,on='관광지명',how='inner')

In [4]:
import numpy as np
score = []

for i in range(len(df)):
    if pd.isna(df.iloc[i]['stars']):
        star = 0
    else:
        star = df.iloc[i]['stars'] * 1.2
    
    if pd.isna(df.iloc[i]['reviews']):
        review = 0
    else:
        review = np.log1p(max(df['reviews'])) - np.log1p(df.iloc[i]['reviews']) * 1.1
    
    search = np.log1p(max(df['search_number'])) - np.log1p(df.iloc[i]['search_number'])
    
    if pd.isna(df.iloc[0]['광역지자체별 거주 방문자 비율']):
        visit = 0
    else:
        visit = np.log1p(df.iloc[i]['광역지자체별 거주 방문자 비율']) * 1.1
    time = np.log1p(df.iloc[i]['평균 체류시간']) * 0.8
    navi = np.log1p(max(df['avg'])) - np.log1p(df.iloc[i]['avg'])
    all = star+review+search+visit+time-navi
    if df.iloc[i]['소분류'] == '산':
        score.append(all*0.9)
    elif df.iloc[i]['소분류'] == '사찰':
        score.append(all*0.9)
    else:
        score.append(all)

df['scores'] = score

In [5]:
first = df.sort_values(by='scores',ascending=False)

In [6]:
first.to_csv('./gyeongnam.csv',encoding='utf-8',index=False)

In [24]:
first['관광지명'].tail(25).tolist()

['무학산',
 '몽고정',
 '마산로봇랜드',
 '창동예술촌',
 '창원과학체험관',
 '대광사(창원)',
 '기업사랑공원',
 '정병산',
 '창원올림픽공원',
 '진해드림파크',
 '실리도',
 '창원시립진해박물관',
 '시루봉',
 '창원시립마산박물관',
 '진해해양공원',
 '상상길(창원)',
 '창원의집',
 '에너지환경과학공원에너지전시관',
 '창원시립마산음악관',
 '창원마법의정원',
 '추산근린공원',
 '백월산',
 '수치해안',
 '장어구이거리',
 '삼포항']

In [25]:
first['관광지명'].head(25).tolist()

['마금산원탕',
 '장복산조각공원',
 '성흥사계곡',
 '주기철목사기념관',
 '주남저수지(철새도래지)',
 '본포수변생태공원',
 '산호공원(용마공원)시(詩)의거리',
 '대산플라워랜드',
 '마산향교',
 '창원봉곡시장',
 '창원편백치유의숲',
 '저도콰이강의다리(스카이워크)',
 '속천항',
 '진해현관아및객사유지',
 '사화공원',
 '가포수변공원',
 '마산오동동아구찜거리',
 'CECO창원컨벤션센터',
 '대방녹지공원',
 '웅천도요지전시관',
 '불모산노을전망대',
 '창원성산패총',
 '북면수변생태공원',
 '창원역사민속관',
 '명서전통시장']

In [13]:
import folium

gyeongnam_map = folium.Map(location=[35.2279728,128.681882],zoom_start=11)

In [14]:
for i in range(len(first)):
    if i == 30:
        break
    folium.Marker(
            [first.iloc[i]['latitude'],first.iloc[i]['longitude']],
            popup=folium.Popup(first.iloc[i]['관광지명'],maxWidth=300),
            fill_opacity=0.5,
            icon=folium.Icon(color='blue')            
            ).add_to(gyeongnam_map)

In [ ]:
gyeongnam_map

==================================================================================================

==================================================================================================

==================================================================================================

In [1]:
import pandas as pd
import os

In [28]:
# 파일이 있는 디렉토리 설정
directory = "./경남/대중교통"

# 모든 CSV 파일 목록 가져오기
file_list = [file for file in os.listdir(directory) if file.endswith('.csv')]

# 모든 CSV 파일을 담을 빈 DataFrame 생성
combined_df = pd.DataFrame()

# 각 CSV 파일을 순회하면서 데이터를 합치기
for file in file_list:
    file_path = os.path.join(directory, file)
    
    # 관광지 명 추출
    landmark_name = file.split('_')[1]  # 파일명에서 '_'를 기준으로 분리하여 첫 번째 요소를 관광지 명으로 사용
    landmark_name = landmark_name.replace('.csv', '')
    
    # CSV 파일을 DataFrame으로 읽기
    df = pd.read_csv(file_path, encoding="cp949")
    
    # 관광지 명 컬럼 추가
    df['관광지명'] = landmark_name
    
    # 현재 파일의 데이터를 합치기
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# 결과를 CSV 파일로 저장하기
# combined_df.to_csv('combined_data.csv', index=False, encoding="utf-8")

# print("합쳐진 데이터를 combined_data.csv로 저장했습니다.")

In [29]:
total_column = []
for i in range(len(combined_df)):
    total = combined_df.iloc[i][6:-2].sum()
    total_column.append(total)

combined_df['total'] = total_column

In [30]:
# 관광지명으로 그룹화하여 총 승객수를 구하고 새로운 컬럼으로 추가
total_passengers_by_landmark = combined_df.groupby('관광지명')['total'].sum().reset_index()
total_passengers_by_landmark.rename(columns={'total': '총 승객수'}, inplace=True)

# 기존 데이터프레임에 총 승객수 컬럼 추가
df = pd.merge(combined_df, total_passengers_by_landmark, on='관광지명', how='left')

In [31]:
bus = df.groupby('관광지명')[['노선']].nunique().reset_index()

In [32]:
new_df = df[['관광지명', '총 승객수']]
new_df = new_df.drop_duplicates()

In [33]:
new_df = pd.merge(new_df,bus,on='관광지명',how='inner')

In [34]:
for i in range(len(new_df)):
    if new_df.iloc[i]['관광지명'] in first.head(25)['관광지명'].tolist():
        new_df.loc[i,'구분'] = '숨은 명소'
    else:
        new_df.loc[i,'구분'] = '일반 명소'

In [42]:
new_df.to_csv('./경남/bus_info.csv',encoding='utf-8',index=False)

In [35]:
print(len(new_df))
print(len(new_df[new_df['구분']=='일반 명소']))
print(len(new_df[new_df['구분']=='숨은 명소']))


40
20
20


In [36]:
new_df[new_df['구분']=='일반 명소'].sort_values(by='총 승객수',ascending=False)

,관광지명,총 승객수,노선,구분
29,창원시립진해박물관,43191,14,일반 명소
4,에너지환경과학공원에너지전시관,14007,11,일반 명소
0,몽고정,11226,53,일반 명소
9,창원시립마산박물관,10425,11,일반 명소
21,상상길(창원),8794,13,일반 명소
22,창동예술촌,8794,13,일반 명소
14,기업사랑공원,4253,18,일반 명소
34,창원과학체험관,4253,18,일반 명소
35,창원올림픽공원,4253,18,일반 명소
26,창원시립마산음악관,4062,5,일반 명소


In [37]:
new_df[new_df['구분']=='숨은 명소'].sort_values(by='총 승객수',ascending=True)

,관광지명,총 승객수,노선,구분
16,대산플라워랜드,17,1,숨은 명소
39,본포수변생태공원,23,2,숨은 명소
6,성흥사계곡,30,1,숨은 명소
5,대방녹지공원,83,2,숨은 명소
15,속천항,87,2,숨은 명소
18,북면수변생태공원,171,5,숨은 명소
25,사화공원,250,1,숨은 명소
2,가포수변공원,275,6,숨은 명소
7,장복산조각공원,286,6,숨은 명소
28,주남저수지(철새도래지),306,5,숨은 명소


In [38]:
from scipy import stats

test_names = ['총 승객수','노선']

p = new_df[new_df['구분'] == '일반 명소'][['총 승객수','노선']]
up = new_df[new_df['구분'] == '숨은 명소'][['총 승객수','노선']]

for test_name in test_names:
    statistic,p_value = stats.levene(p[test_name], up[test_name])
    if p_value > 0.05:
        print(f"{test_name} p-value: {p_value}, 등분산 가정 만족")
    else:
        print(f"{test_name} p-value: {p_value}, 이분산 가정 만족")

총 승객수 p-value: 0.0827681788949117, 등분산 가정 만족
노선 p-value: 0.5569778089684754, 등분산 가정 만족


In [39]:
for test_name in test_names[0:1]:
    t_statistic,pvalue = stats.ttest_ind(
        a = p[test_name],
        b = up[test_name],
        alternative = 'two-sided',
        equal_var = True
)
    if pvalue<0.05:
        print(test_name,pvalue,'귀무가설 기각')
    else:
        print(test_name,pvalue,'귀무가설 기각 불가')

총 승객수 0.021621175045603406 귀무가설 기각


In [40]:
for test_name in test_names[1:]:
    t_statistic,pvalue = stats.ttest_ind(
        a = p[test_name],
        b = up[test_name],
        alternative = 'two-sided',
        equal_var = True
)
    if pvalue<0.05:
        print(test_name,pvalue,'귀무가설 기각')
    else:
        print(test_name,pvalue,'귀무가설 기각 불가')

노선 0.22238833297978491 귀무가설 기각 불가


=========================================================================================================

=========================================================================================================

=========================================================================================================

In [12]:
import requests
from bs4 import BeautifulSoup
import time

In [13]:
url = 'https://korean.visitkorea.or.kr/call'
params = {
    'cmd': 'TOUR_CONTENT_LIST_VIEW',
    'month': 'All',
    'areaCode': '36',
    'sigunguCode': '16',
    'tagId': 'All',
    'sortkind': '1',
    'locationx': '0',
    'locationy': '0',
    'page': '1',
    'cnt': '100',
    'typeList': 'Tour',
    'stampId': '1589345b-b030-11ea-b8bd-020027310001'
}

all_data = []  # 모든 페이지의 데이터를 저장할 리스트

while True:
    response = requests.get(url, params=params)
    data = response.json()

    # 받은 데이터에서 필요한 정보 추출
    result_list = data.get('body', {}).get('result', [])
    for item in result_list:
        img_path = item.get('imgPath', '')
        img_url = f'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id={img_path}'
        
        extracted_data = {
            '관광지명': item.get('TITLE', ''),
            # 'catch': item.get('catchPhrase', ''),
            # 'addr1': item.get('addr1', ''),
            'img': img_url,
            # 'cotid' : item.get('cotId', ''),
            # 'areaCode' : item.get('areaCode', ''),
            # 'cat1' : item.get('cat1', ''),
            # 'cat2' : item.get('cat2', '')
        }
        
        # cotid = extracted_data['cotid']
        # cat1 = extracted_data['cat1']
        # cat2 = extracted_data['cat2']
        # areaCode = extracted_data['areaCode']
        
        # new_url = f'https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid={cotid}&big_category={cat1}&mid_category={cat2}&big_area={areaCode}'
        # response = requests.get(new_url)
        # html = response.text
        # soup = BeautifulSoup(html,'html.parser')
        # description = soup.find('div',class_='blind').text
        # description = description.replace('\n','').replace('\t','')
        # extracted_data['description'] = description

        all_data.append(extracted_data)
    
    # 다음 페이지로 이동
    # if params['page'] in range(0,1000,100):
    #     print('time sleep')
    #     time.sleep(200)
    # else:
    
    print(params['page'])
    params['page'] = str(int(params['page']) + 1)
    

    # 만약 더 이상 페이지가 없으면 반복문 종료
    if not result_list or len(all_data) == 430:
        break

1
2
3
4
5


In [3]:
import pandas as pd

g = pd.read_csv('./data/gyeongnam.csv')
gg = pd.read_csv('./경남/basic_info.csv')

In [8]:
gg = gg.drop_duplicates('관광지명')

In [10]:
g = pd.merge(g,gg[['관광지명','catch','description']],on='관광지명',how='left')

In [14]:
df = pd.DataFrame(all_data)

In [15]:
df

,관광지명,img
0,대풍삼대육곳간,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
1,브라운도트호텔 마산오동동점,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
2,대산미술관,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
3,블랙스톤 벨포레 목장,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
4,창원실내수영장,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
...,...,...
425,월영대,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
426,도남서원,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
427,창원향교,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
428,삼정자동마애불,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...


In [16]:
from tqdm import tqdm
import re

tqdm.pandas()
df['관광지명'] = df['관광지명'].progress_apply(lambda x: x.replace(' ',''))

pattern = re.compile(r'\([^)]*\)')
df['관광지명'] = df['관광지명'].apply(lambda x: re.sub(pattern, '', x))

100%|██████████| 430/430 [00:00<00:00, 429928.66it/s]


In [18]:
g = pd.merge(g,df,on='관광지명',how='left')

In [43]:
g[g.img.isna()]

,관광지명,주소_x,평균 체류시간,stars,reviews,광역지자체별 거주 방문자 비율,search_number,sum,avg,주소_y,대분류,중분류,소분류,longitude,latitude,scores,catch,description,img
32,해양드라마세트장('역적'촬영지해양드라마세트장),경상남도 창원시 마산합포구,116,4.0,1366.0,72.8,6330,11168.0,1015.272727,경상남도 창원시 마산합포구 구산면 석곡리 산183-2,인문(문화/예술/역사),체험관광지,이색체험,128.543852,35.111143,19.218802,가시대 해안마을 모습을 재현한 세트장,해양드라마세트장은 타임머신을 타고 가야 시대로 거슬러 올라가는 여행지이다. 마산의 ...,NaN


In [45]:
g.loc[82,'img'] = df.iloc[27]['img']
g.loc[45,'img'] = df.iloc[133]['img']
g.loc[60,'img'] = df.iloc[36]['img']
g.loc[6,'img'] = df.iloc[351]['img']
g.loc[92,'img'] = df.iloc[256]['img']

g.loc[48,'img'] = df.iloc[20]['img']
g.loc[37,'img'] = df.iloc[376]['img']
g.loc[61,'img'] = df.iloc[44]['img']
g.loc[72,'img'] = df.iloc[324]['img']
g.loc[42,'img'] = df.iloc[7]['img']

g.loc[68,'img'] = df.iloc[84]['img']
g.loc[11,'img'] = df.iloc[382]['img']
g.loc[4,'img'] = df.iloc[352]['img']
g.loc[70,'img'] = df.iloc[342]['img']
g.loc[85,'img'] = df.iloc[109]['img']

g.loc[32,'img'] = df.iloc[119]['img']

In [51]:
g = g.drop(['주소_x','sum'],axis=1).rename(columns={'주소_y':'주소'}).drop_duplicates('관광지명')

In [53]:
g.to_csv('./data/gyeongnam.csv',encoding='utf-8',index=False)